In [ ]:
# DAT640 - Assignment 1b - Preprocessing
# Name:    Rabbir Bin Rabbani
# ID:      247988
# Team:    004

In [2]:
# This is not the main code for the assignment, the main code is named 'assignemnt_1b', same as the assignment.
# This is a preprocessing file I made since it takes around 20 minutes to process the data and I put it in one csv file to save time on my part.

In [3]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import email
import re
import os
from collections import Counter

# define wordlist and stopwords in English.
stopwords = set(nltk.corpus.stopwords.words('english'))
wordlists = set(nltk.corpus.words.words())

from IPython.display import clear_output # Using IPython.display.clear_output to clear the output of a cell.

In [4]:
def process_email_data(file_path):
    print(file_path, end=" ")
    email_body, email_subject = "", ""
    
    mail = email.message_from_string(open(main_file_path+file_path, encoding = 'ISO-8859-1').read())
        
    # Get E-mail subject
    email_subject = mail['subject']
    
    # Get E-mail body
    if mail.is_multipart():
        for payload in mail.get_payload():
            # if payload.is_multipart() then.. 
            email_body = payload.get_payload()
    else:
        email_body = mail.get_payload()
    
    # If body has no value
    if type(email_body) is not str:
        # If body has no value then make it an empty string
        if not email_body:
            email_body = ""
        # If body was multipart then the payload will be 
        # returned as an array with the first index as 
        # the actual message in the mail
        elif type(email_body) is list:
            email_body = email_body[0].as_string()
    
    if type(email_subject) is not str:
        if not email_subject:
            email_subject = ""

    full_email = email_subject + email_body
    print("Stage 1", end="-")
    
    # Remove numbers and all special characters except space
    full_email = re.sub(r'[^a-zA-Z]', ' ', full_email).lower()
    print("Stage 2", end="-")
    
    words = nltk.word_tokenize(full_email)    
        
    # Remove stopwords
    words = ' '.join([word for word in words if word not in stopwords])
    print("Stage 3", end="-")
    
    # remove words that are not part of the english dictionary - using nltk tokenizer.
    words = [word for word in words if word.lower() in wordlists or not word.isalpha()]
    print("Stage 4", end="-")
    
    # Remove single letter words like 'b', 'd', etc..
    full_email = re.sub(r"\b[a-zA-Z]\b", " ", full_email)
    print("Done.")
    
    # The 3 extra features
    subj_len = len(email_subject.split())
    body_len = len(email_body.split())
    hasReturnPath = 1 if "Return-Path" in mail else 0
    
    print(subj_len, body_len, hasReturnPath)
    
    clear_output()
    return (file_path, full_email, subj_len, body_len, hasReturnPath)

In [5]:
main_file_path = 'data/' # Change this to main data directory (keep the ending '/')

In [6]:
data = pd.read_csv(main_file_path+'train/labels.csv')

### Preprocess the Train mail

In [8]:
data[['Id','email','subject_length', 'body_length', 'hasReturnPath']] = data.apply(lambda row: process_email_data(row['Id']), axis=1).apply(pd.Series)

In [9]:
# Save dataset to drive (Preprocessing takes a long time)
data[['Id','email','subject_length', 'body_length', 'hasReturnPath', 'Label']].to_csv("processed_train_dataset.csv", index=False)

### Preprocess the Test mail

In [10]:
# Get test datapaths
path = main_file_path + 'test'
file_paths = pd.Series(['test/' + filename + '/' + fname for filename in os.listdir(path) for fname in os.listdir(main_file_path + 'test/' + filename)])

In [11]:
test_data = file_paths.apply(lambda row: process_email_data(row)).apply(pd.Series)
test_data.columns = ['Id','email','subject_length', 'body_length', 'hasReturnPath']
test_data.head()

,Id,email,subject_length,body_length,hasReturnPath
0,test/000/001,ngx failover plan hi chris tonight we are...,3,101,1
1,test/000/007,internet advertising agreementmark am wor...,3,97,1
2,test/000/009,fmpa oil invoicemark and charlie fmpa is rea...,3,89,1
3,test/000/012,per eric moon attached you will find the st...,0,27,1
4,test/000/019,fwd truly irish return path comein...,3,515,0


In [14]:
test_data.to_csv("processed_test_dataset.csv", index=False)

In [13]:
test_data.head()

,Id,email,subject_length,body_length,hasReturnPath
0,test/000/001,ngx failover plan hi chris tonight we are...,3,101,1
1,test/000/007,internet advertising agreementmark am wor...,3,97,1
2,test/000/009,fmpa oil invoicemark and charlie fmpa is rea...,3,89,1
3,test/000/012,per eric moon attached you will find the st...,0,27,1
4,test/000/019,fwd truly irish return path comein...,3,515,0
